## 1. Load raw data

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [10]:
df_daily_rent = pd.read_csv("../data/raw/daily_rent_detail.csv")
df_weather = pd.read_csv("../data/raw/weather.csv")

C:\Users\Hao Le\AppData\Local\Temp\ipykernel_34996\1505384709.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_daily_rent = pd.read_csv("../data/raw/daily_rent_detail.csv")


# 2. Data cleaning

In [11]:
df_daily_rent_copy = df_daily_rent.copy()
df_weather_copy = df_weather.copy()

# 3. Data Integration

In [12]:
import pandas as pd

# 1. Trích xuất cột cần thiết
df_weather_needs = df_weather_copy[["datetime", "icon"]].copy()
df_daily_rent_needs = df_daily_rent_copy[["started_at"]].copy()

# 2. TẠO CỘT 'datetime' CHO BẢNG RENT 
# Chuyển đổi 'started_at' sang datetime, sau đó lấy phần ngày (YYYY-MM-DD) để khớp với file weather
df_daily_rent_needs['datetime'] = pd.to_datetime(df_daily_rent_needs['started_at'], format='mixed').dt.strftime('%Y-%m-%d')

# (Tùy chọn) Đảm bảo bảng Weather cũng chuẩn format string YYYY-MM-DD để so khớp chính xác
df_weather_needs['datetime'] = pd.to_datetime(df_weather_needs['datetime']).dt.strftime('%Y-%m-%d')

# 3. Merge (Bây giờ cả 2 bảng đều đã có cột 'datetime')
df_final = pd.merge(df_daily_rent_needs, df_weather_needs, how="left", on="datetime")

# Kiểm tra kết quả
print(df_final.head())

            started_at    datetime               icon
0  2020-05-30 17:25:29  2020-05-30               rain
1  2020-05-09 14:42:04  2020-05-09  partly-cloudy-day
2  2020-05-24 17:27:19  2020-05-24             cloudy
3  2020-05-27 15:29:52  2020-05-27  partly-cloudy-day
4  2020-05-31 14:06:03  2020-05-31  partly-cloudy-day


In [13]:
# Check if missing data after merging
assert df_daily_rent.shape[0] == df_final.shape[0]

In [14]:
# Save to
df_final.to_csv("../data/raw/final.csv", index=False)

# 4. Feature engineering

In [15]:
df_temp = pd.to_datetime(df_final["started_at"], format="mixed")

df_final["time"] = pd.DataFrame(df_temp.dt.strftime("%H:%M"))
df_final["dow"] =pd.DataFrame(df_temp.dt.day_name())

In [16]:
df_final.head()

,started_at,datetime,icon,time,dow
0,2020-05-30 17:25:29,2020-05-30,rain,17:25,Saturday
1,2020-05-09 14:42:04,2020-05-09,partly-cloudy-day,14:42,Saturday
2,2020-05-24 17:27:19,2020-05-24,cloudy,17:27,Sunday
3,2020-05-27 15:29:52,2020-05-27,partly-cloudy-day,15:29,Wednesday
4,2020-05-31 14:06:03,2020-05-31,partly-cloudy-day,14:06,Sunday


In [17]:
# --- PHÂN LOẠI GIỜ TRONG NGÀY --- #

# 1. CHUẨN HÓA CỘT 'time' VỀ DẠNG DATETIME
df_final['time'] = pd.to_datetime(df_final['time'], format='mixed')

# 2. TẠO CỘT GIỜ (0-23)
df_final['hour'] = df_final['time'].dt.hour

# 3. HÀM PHÂN LOẠI (Cao điểm, Thường, Ngoài giờ)
def phan_loai_gio(h):
    # Cao điểm: 7h-9h (tức 7, 8) VÀ 17h-19h (tức 17, 18)
    if (7 <= h < 9) or (17 <= h < 19):
        return 'Giờ cao điểm'
    
    # Ngoài giờ: Sau 19h tối hoặc Trước 7h sáng
    elif (h >= 19) or (h < 7):
        return 'Ngoài giờ'
    
    # Giờ thường: Các giờ còn lại (9h-16h)
    else:
        return 'Giờ thường'

# 4. ÁP DỤNG VÀO DỮ LIỆU
df_final['group_time'] = df_final['hour'].apply(phan_loai_gio)

# --- KIỂM TRA KẾT QUẢ ---
print("✅ Đã thêm cột 'group_time' thành công!")
print("Thống kê số lượng:")
print(df_final['group_time'].value_counts())

print("\nXem 5 dòng đầu tiên:")
print(df_final[['started_at', 'hour', 'group_time', 'datetime', 'icon', 'time', 'dow']].head())

✅ Đã thêm cột 'group_time' thành công!
Thống kê số lượng:
group_time
Giờ thường      7675666
Giờ cao điểm    4482488
Ngoài giờ       3928518
Name: count, dtype: int64

Xem 5 dòng đầu tiên:
            started_at  hour    group_time    datetime               icon  \
0  2020-05-30 17:25:29    17  Giờ cao điểm  2020-05-30               rain   
1  2020-05-09 14:42:04    14    Giờ thường  2020-05-09  partly-cloudy-day   
2  2020-05-24 17:27:19    17  Giờ cao điểm  2020-05-24             cloudy   
3  2020-05-27 15:29:52    15    Giờ thường  2020-05-27  partly-cloudy-day   
4  2020-05-31 14:06:03    14    Giờ thường  2020-05-31  partly-cloudy-day   

                 time        dow  
0 2025-12-29 17:25:00   Saturday  
1 2025-12-29 14:42:00   Saturday  
2 2025-12-29 17:27:00     Sunday  
3 2025-12-29 15:29:00  Wednesday  
4 2025-12-29 14:06:00     Sunday  


In [44]:
# --- PHÂN LOẠI NGÀY TRONG TUẦN --- #

import numpy as np

# 1. Chuẩn hóa cột 'dow' về chữ thường (lowercase)
# .astype(str) để đảm bảo không bị lỗi nếu có dữ liệu lạ không phải chữ
df_final['dow'] = df_final['dow'].astype(str).str.lower().str.strip()

# 2. Định nghĩa danh sách ngày cuối tuần 
nhom_cuoi_tuan = ['saturday', 'sunday']

# 3. Tạo cột mới 'day_type' (Loại ngày)
df_final['day_type'] = np.where(
    df_final['dow'].isin(nhom_cuoi_tuan), 
    'Ngày cuối tuần', 
    'Ngày thường'
)

# --- KIỂM TRA KẾT QUẢ ---
print("✅ Đã thêm cột 'day_type' thành công!")
print("Thống kê số lượng:")
print(df_final['day_type'].value_counts())

print("\nXem thử 5 dòng đầu:")
print(df_final[['started_at', 'hour', 'group_time', 'datetime', 'icon', 'time', 'dow', 'day_type']].head())

✅ Đã thêm cột 'day_type' thành công!
Thống kê số lượng:
day_type
Ngày thường       11234701
Ngày cuối tuần     4851971
Name: count, dtype: int64

Xem thử 5 dòng đầu:
            started_at  hour    group_time    datetime               icon  \
0  2020-05-30 17:25:29    17  Giờ cao điểm  2020-05-30               rain   
1  2020-05-09 14:42:04    14    Giờ thường  2020-05-09  partly-cloudy-day   
2  2020-05-24 17:27:19    17  Giờ cao điểm  2020-05-24             cloudy   
3  2020-05-27 15:29:52    15    Giờ thường  2020-05-27  partly-cloudy-day   
4  2020-05-31 14:06:03    14    Giờ thường  2020-05-31  partly-cloudy-day   

                 time        dow        day_type  
0 2025-12-29 17:25:00   saturday  Ngày cuối tuần  
1 2025-12-29 14:42:00   saturday  Ngày cuối tuần  
2 2025-12-29 17:27:00     sunday  Ngày cuối tuần  
3 2025-12-29 15:29:00  wednesday     Ngày thường  
4 2025-12-29 14:06:00     sunday  Ngày cuối tuần  


In [22]:
# Save to
df_final.to_csv("../data/raw/final.csv", index=False)

# 5. EDA

In [42]:
df_daily_rent.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,946D42AD89539210,docked_bike,2020-05-30 17:25:29,2020-05-31 18:25:22,Anacostia Library,31804,11th & H St NE,31614.0,38.865784,-76.978400,38.899983,-76.991383,casual
1,CC46FAAB662B8613,docked_bike,2020-05-09 14:42:04,2020-05-09 15:06:33,10th & E St NW,31256,21st St & Constitution Ave NW,31261.0,38.895914,-77.026064,38.892459,-77.046567,member
2,72F00B2FB833D6ED,docked_bike,2020-05-24 17:27:19,2020-05-24 17:43:51,Connecticut Ave & Newark St NW / Cleveland Park,31305,12th & U St NW,31268.0,38.934267,-77.057979,38.916787,-77.028139,member
3,4DFBE6AED989DF35,docked_bike,2020-05-27 15:29:52,2020-05-27 15:47:13,Connecticut Ave & Newark St NW / Cleveland Park,31305,14th & Belmont St NW,31119.0,38.934267,-77.057979,38.921074,-77.031887,casual
4,1AAFE6B4331AB9DF,docked_bike,2020-05-31 14:06:03,2020-05-31 14:30:30,Georgia Ave & Morton St NW,31419,17th & K St NW,31213.0,38.932128,-77.023500,38.902760,-77.038630,casual


In [36]:
# check missing value
import matplotlib.pyplot as plt

if len(missing_only) > 0:
    plt.figure(figsize=(10, 6))
    
    # Vẽ biểu đồ ngang
    ax = missing_only.plot(kind="barh", color="salmon", edgecolor='black')
    
    plt.title("Thống kê dữ liệu bị khuyết (Missing Values)", fontsize=14)
    plt.xlabel("Số lượng dòng bị thiếu (NaN)", fontsize=12)
    plt.ylabel("Tên cột", fontsize=12)
    plt.grid(axis='x', linestyle='--', alpha=0.7)

    # 3. THÊM CON SỐ CỤ THỂ LÊN BIỂU ĐỒ
    for index, value in enumerate(missing_only):
        # Tính phần trăm
        percent = (value / len(df_final)) * 100
        label = f"{value} ({percent:.2f}%)"
        
        # Ghi chữ lên biểu đồ
        plt.text(value, index, label, va='center', fontsize=10, color='black', fontweight='bold')

    plt.tight_layout()
    plt.show()
    
    print("Chi tiết các cột bị thiếu:")
    print(missing_only)

else:
    print("✅ Không có giá trị nào bị thiếu (NaN).")


✅ Không có giá trị nào bị thiếu (NaN).


In [41]:
# Check duplicated Values by col 'started_at'
duplicate_count = df_final.duplicated(subset=['started_at']).sum()

print(f"Số lượng dòng bị trùng: {duplicate_count}")
print(f"Tổng số dòng dữ liệu: {len(df_final)}")

Số lượng dòng bị trùng: 1404757
Tổng số dòng dữ liệu: 16086672


In [45]:
df_final.drop_duplicates(subset=['started_at'], keep='first')

,started_at,datetime,icon,time,dow,hour,group_time,day_type
0,2020-05-30 17:25:29,2020-05-30,rain,2025-12-29 17:25:00,saturday,17,Giờ cao điểm,Ngày cuối tuần
1,2020-05-09 14:42:04,2020-05-09,partly-cloudy-day,2025-12-29 14:42:00,saturday,14,Giờ thường,Ngày cuối tuần
2,2020-05-24 17:27:19,2020-05-24,cloudy,2025-12-29 17:27:00,sunday,17,Giờ cao điểm,Ngày cuối tuần
3,2020-05-27 15:29:52,2020-05-27,partly-cloudy-day,2025-12-29 15:29:00,wednesday,15,Giờ thường,Ngày thường
4,2020-05-31 14:06:03,2020-05-31,partly-cloudy-day,2025-12-29 14:06:00,sunday,14,Giờ thường,Ngày cuối tuần
...,...,...,...,...,...,...,...,...
16086667,2024-08-01 08:10:22.293,2024-08-01,partly-cloudy-day,2025-12-29 08:10:00,thursday,8,Giờ cao điểm,Ngày thường
16086668,2024-08-08 10:05:21.938,2024-08-08,rain,2025-12-29 10:05:00,thursday,10,Giờ thường,Ngày thường
16086669,2024-08-03 16:29:32.252,2024-08-03,rain,2025-12-29 16:29:00,saturday,16,Giờ thường,Ngày cuối tuần
16086670,2024-08-03 02:49:45.661,2024-08-03,rain,2025-12-29 02:49:00,saturday,2,Ngoài giờ,Ngày cuối tuần
